In [ ]:
import wfdb

import scipy.io

import numpy as np
import pandas as pd

#from skimage import transform
#from scipy.ndimage import zoom
import resampy
from tqdm.auto import tqdm
from pathlib import Path

#thew
from ishneholterlib import Holter
from .stratify import stratify,stratify_batched

#ribeiro
import h5py
import datetime

#from clinical_ts.misc_utils import *
from .timeseries_utils import *

In [ ]:
def map_and_filter_labels(df,min_cnt,lbl_cols):
    #filter labels
    def select_labels(labels, min_cnt=10):
        lbl, cnt = np.unique([item for sublist in list(labels) for item in sublist], return_counts=True)
        return list(lbl[np.where(cnt>=min_cnt)[0]])
    df_ptb_xl = df.copy()
    lbl_itos_ptb_xl = {}
    for selection in lbl_cols:
        if(min_cnt>0):
            label_selected = select_labels(df_ptb_xl[selection],min_cnt=min_cnt)
            df_ptb_xl[selection+"_filtered"]=df_ptb_xl[selection].apply(lambda x:[y for y in x if y in label_selected])
            lbl_itos_ptb_xl[selection+"_filtered"] = np.array(sorted(list(set([x for sublist in df_ptb_xl[selection+"_filtered"] for x in sublist]))))
            lbl_stoi = {s:i for i,s in enumerate(lbl_itos_ptb_xl[selection+"_filtered"])}
            df_ptb_xl[selection+"_filtered_numeric"]=df_ptb_xl[selection+"_filtered"].apply(lambda x:[lbl_stoi[y] for y in x])
        #also lbl_itos and ..._numeric col for original label column
        lbl_itos_ptb_xl[selection]= np.array(sorted(list(set([x for sublist in df_ptb_xl[selection] for x in sublist]))))
        lbl_stoi = {s:i for i,s in enumerate(lbl_itos_ptb_xl[selection])}
        df_ptb_xl[selection+"_numeric"]=df_ptb_xl[selection].apply(lambda x:[lbl_stoi[y] for y in x])
    return df_ptb_xl, lbl_itos_ptb_xl